[OpenCV Tutorial](https://opencv24-python-tutorials.readthedocs.io/en/stable/py_tutorials/py_calib3d/py_calibration/py_calibration.html) - How to Calculate Reprojectio Error

We need to find five parameters, known as distortion coefficients given by: (k1 k2 p1 p2 k3) <br>
In addition to this, we need to find a few more information, like intrinsic and extrinsic parameters of a camera.

Summary
1. For stereo applications, these distortions need to be corrected first. 
2. To find all these parameters, what we have to do is to provide some sample images of a well defined pattern (eg, chess board). We find some specific points in it ( square corners in chess board). 3D points are called object points and 2D image points are called image points.
3. We know its coordinates in real world space and we know its coordinates in image. 
4. With these data, some mathematical problem is solved in background to get the distortion coefficients.
5. We need atleast 10 test patterns.

Setup

In [1]:
import numpy as np
import cv2
import glob

- criteria： it is the iteration termination criterion when this condition is met，algorithm iteration stops, actually，it should be a tuple of three arguments they are (type，max_iter，epsilon）

In [2]:
# Define termination criteria = ( type, max_iter = 30 , epsilon = 0.001 )
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

In [ ]:
# Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

In [4]:
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane

# creates a list of strings of filepaths to image files
images = sorted(glob.glob('images/left??.jpg'))

In [5]:
# image
img = cv2.imread(images[0])
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# Find the chess board corners
marker_detected, corners = cv2.findChessboardCorners(gray, (7,6), None)

# Draw and display the corners
objpoints.append(objp) #gerei
corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria) # aumentar acuracia
imgpoints.append(corners2) #gerei + aumentei acuracia

In [ ]:
# display image
img = cv2.drawChessboardCorners(img, (7,6), corners2, marker_detected)
cv2.imshow('img',img)
cv2.waitKey(500)

Calibration

In [6]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

**Undistortion** <br>
We have got what we were trying:
- Object and image points
- Camera calibration (intrinsics and extrinsics)

Now we can take an image and undistort it.

In [12]:
# We can refine the camera matrix based on a free scaling parameter using cv2.getOptimalNewCameraMatrix()
img = cv2.imread('images/left12.jpg')
h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

In [14]:
# undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibresult.png',dst)

True

**Reprojection Error** <br>
Reprojection error gives a good estimation of just how exact is the found parameters. This should be as close to zero as possible. Given the intrinsic, distortion, rotation and translation matrices: 
1. We first transform the object point to image point using cv2.projectPoints(). 
2. Then we calculate the absolute norm between what we got with our transformation (ground truth) and the corner finding algorithm.
3. Find the average error we calculate the arithmetical mean of the errors calculate for all the calibration images.

In [17]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error

print ("Reprojection Error: ", mean_error/len(objpoints))

Reprojection Error:  0.022552185443330906
